In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

load_dotenv()
directory = os.getenv("path")

model = AffinityPropagation()

In [64]:
# selected attributes
selected_attr = pd.read_excel(directory + '/selected_attributes.xlsx', sheet_name='Sheet1')
selected_attr

,surfical_geology,soil,lulc,terrain,crop inventories,year river flow metrics,seasonal river flow metrics
0,Ground moraine,median_topsoil_sand,grassland,range_elevation,winterwheat,Specific-YR-MaxFlow,Specific-GM-MinFlow
1,Lacustrine,std_topsoil_silt,wetland,std_elevation,soyabeans,Specific-YR-Q95,Specific-GM-Q5
2,End moraine,median_topsoil_silt,water,mean_elevation,oats,YR-CVQ,Specific-NGM-MinFlow
3,Alluvium,std_topsoil_clay,urban,median_elevation,tomatoes,Specific-YR-MinFlow,Specific-GM-Max
4,NaN,NaN,NaN,NaN,NaN,YR-DOY-MaxFlow,NaN


In [83]:
# most shared attributes
func_yr_selected = selected_attr["year river flow metrics"].dropna().tolist()
func_sz_selected = selected_attr["seasonal river flow metrics"].dropna().tolist()
func_selected_attr = func_yr_selected + func_sz_selected

func_path = os.path.join(directory, "functional_attributes/133_riverflow")
func_files = os.listdir(func_path)
func_files = [file for file in func_files if file.endswith(".csv")]

func_ap_res = {}

for file in func_files:
    df = pd.read_csv(os.path.join(func_path, file), index_col=0)
    df = df.loc[:, df.columns.str.contains('|'.join(func_selected_attr))]
    df = df.fillna(df.median())

    # ap
    model.fit(df)
    labels = model.labels_
    labels = labels + 1
    func_ap_res[file.split(".")[0]] = labels

# stations
# stations = pd.read_csv(directory + "/functional_attributes/133_riverflow/", index_col=0)

func_ap_res = pd.DataFrame(func_ap_res)
func_ap_res = func_ap_res.reindex(sorted(func_ap_res.columns), axis=1)
func_ap_res = func_ap_res.set_index(df.index)

# out_put_path = directory + "/ap_results"
# func_ap_res.to_csv(os.path.join(out_put_path, "func_ap_res.csv"))

func_ap_res_dict = {col: len(func_ap_res[col].unique()) for col in func_ap_res.columns}
func_ap_res_dict


{'2011': 18,
 '2012': 18,
 '2013': 19,
 '2014': 18,
 '2015': 23,
 '2016': 20,
 '2017': 20,
 '2018': 24,
 '2019': 23,
 '2020': 22}